In [ ]:
!pip install -r /kaggle/input/aaaaaa/requirements.txt

In [ ]:
!pip install captcha

Dataset

In [ ]:
from torchtext.vocab import build_vocab_from_iterator
from collections import Counter
from rich import console

console = console.Console()


def get_default_vocab():
    cab_a = r'abcdefghijklmnopqrstuvwxyz'
    cab_A = r'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
    cab_0 = r'0123456789'
    cab_t = cab_a + cab_0

    cab = [str(word) for word in cab_t]
    cab = build_vocab_from_iterator(cab, specials=['<BOS>', '<EOS>'], max_tokens=len(cab) + 2)
    return cab

In [ ]:
!pwd
! rm -r ./captcha.csv

In [ ]:
import os.path
import pandas as pd
from captcha import image
import random

dir_to_save = r'./gener_captchas'
cab_list = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
            'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u',
            'v', 'w', 'x', 'y', 'z',
            # 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U',
            # 'V',
            # 'W', 'X', 'Y', 'Z'
            ]

width, height = 160, 60
generator = image.ImageCaptcha(width=width, height=height)
file_paths = []
lables = []
if not os.path.exists(dir_to_save):
    os.mkdir(dir_to_save)
for i in range(20000):

    random_str = ''.join([random.choice(cab_list) for j in range(5)])
    if random_str in lables:
        continue
    img = generator.generate_image(random_str)
    # img =generator.create_noise_curve(img, '#000000')
    file_path = os.path.join(dir_to_save, f'{random_str}_{str(i)}' + '.jpg')
#     print(file_path)
    file_paths.append(file_path)
    lables.append(random_str)
    img.save(file_path)
dataframe = pd.DataFrame(list(zip(file_paths, lables)), columns=['path', 'lable'])
print('done')
dataframe.to_csv(r'./captcha.csv')


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class MyDataSet(Dataset):
    def __init__(self, csv_path, cab):
        super(MyDataSet, self).__init__()
        self.data = pd.read_csv(csv_path)
        self.cab = cab
        self.stoi = self.cab.get_stoi()
        self.itos = self.cab.get_itos()

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        dicts = self.data.iloc[idx].to_dict()
        lable = dicts['lable']
        lable = [word for word in str(lable)]
        lable.insert(0, '<BOS>')
        lable.append('<EOS>')
        lable = map(lambda x: self.stoi[x], lable)
        return {'path': dicts['path'], 'lable': list(lable)}

F_Dateset

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
import pandas as pd


class MyDataSet(Dataset):
    def __init__(self, csv_path, cab):
        super(MyDataSet, self).__init__()
        self.data = pd.read_csv(csv_path)
        self.cab = cab
        self.stoi = self.cab.get_stoi()
        self.itos = self.cab.get_itos()
        self.items = []
        for idx in range(len(self.data)):
#             if idx % 100==0:
#                 print(idx)
            dicts = self.data.iloc[idx].to_dict()
            lable = dicts['lable']
            lable = [word for word in str(lable)]
            lable.insert(0, '<BOS>')
            lable.append('<EOS>')
            lable = map(lambda x: self.stoi[x], lable)
            self.items.append({'path': dicts['path'], 'lable': list(lable)})

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.items[idx]


Model

In [ ]:
!pip install einops

In [ ]:
import torch
import numpy as np
from torch import nn, einsum
from einops import rearrange, repeat, reduce
from rich import console

console = console.Console()
device = 'cuda' if torch.device('cuda') else 'cpu'


class MultiHeadAttention(nn.Module):
    def __init__(self, head_num, emb_d):
        super(MultiHeadAttention, self).__init__()
        self.head_num = head_num
        self.emb_d = emb_d
        self.dk = (emb_d // head_num) ** (1 / 2)
        self.qkv_layer = nn.Linear(emb_d, emb_d * 3, bias=False)
        self.out_attention = nn.Linear(emb_d, emb_d, bias=False)

    def forward(self, x, mask=None, kv=None):
        qkv = self.qkv_layer(x)
        q, k, v = tuple(rearrange(qkv, pattern='b t (h k d) -> k b h t d', h=self.head_num, k=3))
        if kv is not None:
            # decoder 部分
            # kk = repeat(kv, 'b t (d)->b t (d h)', h=self.head_num)
            k = rearrange(kv, 'b t (h d)-> b h t d', h=self.head_num)
            v = k
        energy = einsum('... i d , ... j d -> ... i j', q, k)
        energy = energy / self.dk
        if mask is not None:
            # console.log(mask)
            energy += mask
            # console.log(energy[0][0])
        attention = torch.softmax(energy, dim=-1)
        # console.log(attention[0][0])
        xx = einsum('... i j , ... j d -> ... i d', attention, v)
        xx = rearrange(xx, 'b h t d -> b t (h d)')
        xx = self.out_attention(xx)
        # console.log(xx)
        return xx


In [ ]:
import math

import torch
from torch import nn


class PositionalEncoding(nn.Module):
    """Implement the PE function."""

    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) *
                             -(math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        self.register_buffer('pe', pe)

    def forward(self, x):
        # print(self.pe.shape)
        # print(x.shape)
        x = x + self.pe[:, :x.size(2)]
        return self.dropout(x)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


class EncoderBlock(nn.Module):
    def __init__(self, mlp_d, head_num, emb_d, dropout):
        super(EncoderBlock, self).__init__()

        self.m_attention = MultiHeadAttention(head_num, emb_d)
        self.mlp_layer = MLP(emb_d, mlp_d)
        self.drop_layer = nn.Dropout(dropout)
        self.norm_layer1 = nn.LayerNorm(emb_d)
        self.norm_layer2 = nn.LayerNorm(emb_d)

    def forward(self, x):
        x = self.m_attention(x=x)
        _x = self.drop_layer(x)
        x = _x + x
        x = self.norm_layer1(x)

        x = self.mlp_layer(x)
        x = _x + x
        x = self.norm_layer2(x)
        return x


class MLP(nn.Module):
    def __init__(self, input_d, middle_d):
        super(MLP, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_d, middle_d),
            nn.Dropout(0.1),
            nn.GELU(),
            nn.Linear(middle_d, input_d),
            nn.Dropout(0.1)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
class DecoderBlock(nn.Module):
    def __init__(self, mlp_d, emb_d, dropout, head_num):
        super(DecoderBlock, self).__init__()
        self.masked_attention = MultiHeadAttention(head_num, emb_d)
        self.attention = MultiHeadAttention(head_num, emb_d)
        self.drop_layer = nn.Dropout(dropout)
        self.normal_layer0 = nn.LayerNorm(emb_d)
        self.normal_layer2 = nn.LayerNorm(emb_d)
        self.normal_layer3 = nn.LayerNorm(emb_d)
        self.mlp_layer = MLP(emb_d, mlp_d)

    def forward(self, x, kv, mask):
        mask_attention = self.masked_attention(x=x, mask=mask)
        # console.log(mask_attention)
        _x = self.drop_layer(mask_attention)
        x = _x + x
        x = self.normal_layer0(x)

        multihead_attention = self.attention(x=mask_attention, kv=kv)
        _x = self.drop_layer(multihead_attention)
        x = _x + x
        x = self.normal_layer2(x)

        _x = self.mlp_layer(x)
        x = _x + x
        x = self.normal_layer3(x)
        return x


In [ ]:
from rich import console

console = console.Console()


class Encoder(nn.Module):
    def __init__(self, block_size, mlp_d, head_num, emd_d, dropout):
        super(Encoder, self).__init__()
        self.layers = nn.ModuleList(
            EncoderBlock(mlp_d, head_num, emd_d, dropout) for _ in range(block_size)
        )

    def forward(self, x):
        # b t emb_d
        for layer in self.layers:
            x = layer(x)
        return x


In [ ]:
class Decoder(nn.Module):
    def __init__(self, block_size, mlp_d, head_num, emb_d, dropout):
        super(Decoder, self).__init__()
        self.layers = nn.ModuleList(
            DecoderBlock(mlp_d, emb_d, dropout, head_num) for _ in range(block_size)
        )

    def forward(self, x, mask, enc_out):
        # b t emb_d
        # console.log(x.shape)
        for layer in self.layers:
            x = layer(x=x, mask=mask, kv=enc_out)
        return x


In [ ]:
import torch
from matplotlib import pyplot as plt
from torch.nn import TransformerEncoder
from torchvision.transforms import transforms
from einops import repeat
from rich import console

console = console.Console()
device = 'cuda' if torch.device('cuda') else 'cpu'


class Transformer(nn.Module):
    def __init__(self, block_size, mlp_d, head_num, emb_d, dropout, in_channels, img_x, img_y, patch_d, vocab):
        super(Transformer, self).__init__()
        self.img_w = img_x
        self.img_h = img_y
        self.patch_d = patch_d
        self.in_channels = in_channels
        self.token_d = in_channels * (patch_d ** 2)
        self.token_num = (img_y // patch_d) * (img_x // patch_d)

        self.pos_emb = nn.Parameter(
            torch.randint(low=0, high=5, size=(self.token_num, emb_d), device=device).float())
        nn.init.xavier_normal_(self.pos_emb)
        # self.pos_emb = PositionalEncoding(d_model=emb_d, dropout=dropout, max_len=self.token_num)
        # self.cls_emb = nn.Parameter(torch.randn(1, 1, emb_d, device=device))
        # nn.init.xavier_normal_(self.pos_emb)
        # self.transformer_encoder = nn.TransformerEncoder(num_layers=block_size, norm=True)
        self.encoder = Encoder(block_size, mlp_d, head_num, emb_d, dropout)
        self.decoder = Decoder(block_size, mlp_d, head_num, emb_d, dropout)

        self.emb_layer = nn.Linear(self.token_d, emb_d)
        self.embbding_layper = nn.Embedding(vocab, emb_d)
        self.drop_layer = nn.Dropout(dropout)

        self.dense_layer = nn.Linear(emb_d, vocab)  # self.out_h = nn.LeakyReLU()
        self.out_h = nn.GELU()
        # self.dense_layer1 = nn.Linear(emb_d // 2, vocab)
        # self.sigmod = nn.Tanh()
        self.softmax_layer = nn.LogSoftmax(dim=-1)

    def forward(self, words, enc_out=None, img=None, **kwargs):
        if img is not None:
            # console.log(img.shape)
            # B C H W
            img = rearrange(img, 'b c (patch_x x) (patch_y y) -> b (x y) (patch_x patch_y c)',
                            patch_x=self.patch_d,
                            patch_y=self.patch_d)
            # imm = transforms.ToPILImage()
            # plt.imshow(imm(img[0]))
            # plt.show()
            img = self.emb_layer(img)
            # x: [b t emb_d]
            batch, tokens, _ = img.shape
            # 分类+位置初始化编码
            # cls = repeat(self.cls_emb, 'b ...-> (b batches) ...', batches=batch)
            # img = torch.cat([cls, img], dim=1)

            img += self.pos_emb
            # img = self.pos_emb(img)
            img = self.drop_layer(img)

            enc_out = self.encoder(img)
            # enc_out = self.transformer_encoder(img)
            # console.log(enc_out[0][-1])
        emb = self.embbding_layper(words)

        # mask = torch.triu(torch.ones(len(words[0]), len(words[0])), diagonal=1).bool().cuda()
        mask = nn.Transformer.generate_square_subsequent_mask(len(words[0]), device)
        # console.log(mask)
        next_word = self.decoder(x=emb, enc_out=enc_out, mask=mask)
        # console.log(next_word[0][-1])
        # console.log(next_word.shape)
        out = self.dense_layer(next_word)
        # out = self.out_h(out)
        # out:[b t d]
        # out = self.out_h(out)
        # out = self.dense_layer1(out)
        # out = self.sigmod(out)
        # out = self.softmax_layer(out)
        # console.log(out[0])
        # console.log(torch.argmax(out[0], dim=-1))

        return out, enc_out


In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

Test


In [ ]:
# @title 默认标题文本
import random

import PIL.ImageShow
from PIL import Image
import torch
from torch import optim
from torchvision import transforms
from torch.utils.data import random_split
import matplotlib.pyplot as plt
from tqdm import tqdm
from rich import console
console =console.Console()

trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.ConvertImageDtype(torch.float),
])

cab = get_default_vocab()


def main():
    datas = MyDataSet(r'./captcha.csv', cab)
    # console.log(cab.get_stoi())
    train_size = int(len(datas) * 0.95)
    test_size = len(datas) - train_size
    train_dataset, test_dataset = random_split(datas, [train_size, test_size])
    model = train(train_dataset, test_dataset)
    torch.save(model,r'./aa.pt')



def train(train_set, test_set):
    epochs = 500
    dataloader = DataLoader(train_set, batch_size=60, shuffle=False, drop_last=True, collate_fn=col_fn,
                            num_workers=0)
    # for batch in dataloader:
    #     console.log(batch['lables'].shape)
    #     console.log(batch['imgs'].shape)
    model = Transformer(block_size=4, mlp_d=195, head_num=5, emb_d=180, dropout=0.1, in_channels=1, img_x=60,
                        img_y=160,
                        patch_d=4,
                        vocab=38).to(device)
    print(torch.cuda.device_count(), "GPUs!")
    model = nn.DataParallel(model)
    model = model.cuda()

    ce_loss = nn.CrossEntropyLoss()
    opti = optim.Adam(model.parameters(), lr=0.0007)
    for epoch in tqdm(range(epochs), leave=False):
        loss_epoch = 0
        for batch in dataloader:
            # { lables:[b t],imgs:[b c w h]}
            # console.log(len(batch['lables']))
            # console.log(batch['imgs'].shape)
            # console.log(batch['lables'].shape)
            decoder_input, decoder_tgt = get_decoder_tgt_and_input(batch)
            # console.log(decoder_input[:5])
            # console.log(decoder_tgt[:5])
            # inp:[b t];img:[b c h w]

            # imm = transforms.ToPILImage()
            # plt.imshow(imm(batch['imgs'][0]))
            # plt.show()
            # console.log(batch['imgs'].shape)
            # console.log(get_text_from_vocab(decoder_input[0]))
            # console.log(f"path:{batch['path'][0]};text:{get_text_from_vocab(batch['lables'][0])}")

            out, _ = model(img=batch['imgs'].cuda(), words=decoder_input)
            # out, _ = model(img=torch.randn(batch['imgs'].shape).cuda(),
            #                words=torch.randint(size=decoder_input.shape, low=1, high=39).long().cuda())
            # console.log(pred.shape)
            # console.log(out)
            out = rearrange(out, 'b t d-> (b t) d')
            decoder_tgt = rearrange(decoder_tgt, 'b t->(b t)')
            # console.log(f'inp:{torch.randint(size=decoder_input.shape, low=1, high=39).long().cuda()[1]}')
            # console.log(f'out:{torch.argmax(out, dim=-1)[:10]}')
            # console.log(f'tgt:{decoder_tgt[:10]}')
            loss = ce_loss(input=out, target=decoder_tgt)
            # console.log(decoder_tgt[4].item())
            # console.log(out[4][decoder_tgt[4].item()])
            opti.zero_grad()
            loss.backward()
            opti.step()
            loss_epoch += loss

            # print(loss)
        model.eval()
        test_loss = evaluate(model, test_set=test_set)
        model.train()
        # tqdm.set_description(f'Epoch {epoch + 1}, Loss {loss_epoch / (len(train_set) / 50):.4f}', refresh=True)
        console.log(f'epoch: {epoch + 1};loss: {loss_epoch / (len(train_set) / 50):.4f};test_acc:{test_loss:.4f}')
    return model


def evaluate(model, test_set):
    test_set_loader = DataLoader(test_set, batch_size=50, shuffle=False, drop_last=True, collate_fn=col_fn,
                                 num_workers=0)

    # model.eval()
    acc_count = []
    out_sample = []
    tgt_sample = []
    for batch in tqdm(test_set_loader, leave=False, dynamic_ncols=True):
        decoder_input, decoder_tgt = get_decoder_tgt_and_input(batch)
        out, kv = model(imgs=batch['imgs'], words=decoder_input)
        # out:[b t d],kv:[b t d]

        decoder_tgt = rearrange(decoder_tgt, 'b t->(b t)')
        out = rearrange(out, 'b t d-> (b t) d')

        out = torch.argmax(out, dim=-1)

        equal = torch.eq(out, decoder_tgt)
        # console.log(f'inp:{rearrange(decoder_input, "b t->(b t)")[:5]}')
        out_sample.append(out[:6])
        tgt_sample.append(decoder_tgt[:6])
        # 计算相等元素的个数，返回一个标量
        count = torch.sum(equal) / len(out)
        acc_count.append(count)
    console.log(f'out:{random.choice(out_sample)}')
    console.log(f'tgt:{random.choice(tgt_sample)}')
    return torch.mean(torch.Tensor(acc_count))


# def test_model(model: nn.Module):


def get_decoder_tgt_and_input(batch):
    decoder_input = batch['lables'].cuda()[:, :-1]
    decoder_tgt = batch['lables'].cuda()[:, 1:]
    return decoder_input, decoder_tgt


def col_fn(batch):
    imgs = []
    lables = []
    paths = []
    for one_data in batch:
        img = Image.open(one_data['path']).convert('L')
        # img = Image.open(one_data['path'])
        # console.log(f"path:{one_data['path']};text:{get_text_from_vocab(one_data['lable'])}")
        imgs.append(trans(img))
        # img.show()
        lables.append(one_data['lable'])
        paths.append(one_data['path'])
        # console.log(one_data['lable'])
    return {'imgs': torch.stack([tensor for tensor in imgs]), 'lables': torch.Tensor(lables).long(), 'path': paths}


def get_text_from_vocab(lable):
    mapp = cab.get_itos()
    return [mapp[idx] for idx in lable]


if __name__ == '__main__':
    main()